<a href="https://colab.research.google.com/github/kumarmohit0911/AAA/blob/main/IROC_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import json, os

kaggle_creds = {
    "username": "kumarmohit0911",
    "key": "KGAT_70d781b0a7eed54b863a6bcb5ab50e9c"
}

os.makedirs("/root/.kaggle", exist_ok=True)

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_creds, f)

os.chmod("/root/.kaggle/kaggle.json", 0o600)

print("kaggle.json created successfully")

kaggle.json created successfully


In [4]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



cp: cannot stat 'kaggle.json': No such file or directory


In [5]:
import torch
import torch.nn as nn


In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("stealthtechnologies/rock-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'rock-classification' dataset.
Path to dataset files: /kaggle/input/rock-classification


In [7]:
os.listdir(path)

['Rock Data']

In [8]:
import os
DATA_DIR = '/kaggle/input/rock-classification'
os.listdir(DATA_DIR)

['Rock Data']

In [9]:
import os
DATA_DIR = '/kaggle/input/rock-classification/Rock Data'
os.listdir(DATA_DIR)

['README.dataset.txt', 'README.roboflow.txt', 'valid', 'test', 'train']

In [13]:
import os
train_data_paths  = []
train_labels = []
train_DATA_DIR = '/kaggle/input/rock-classification/Rock Data/train'
# train_dir_list = os.listdir(train_DATA_DIR)
for rock_type in os.listdir(train_DATA_DIR):
  if rock_type == 'Shale-(Mudstone)':
    ShaleMudstone= os.path.join(train_DATA_DIR,'Shale-(Mudstone)')
    for rock_image in os.listdir(ShaleMudstone):
      train_data_paths.append(os.path.join(ShaleMudstone, rock_image))
      train_labels.append(0)
  elif rock_type =='gypsum':
    gypsum = os.path.join(train_DATA_DIR,'gypsum')
    for rock_image in os.listdir(gypsum):
      train_data_paths.append(os.path.join(gypsum,rock_image))
      train_labels.append(1)
  elif rock_type =='chert':
    chert = os.path.join(train_DATA_DIR,'chert')
    for rock_image in os.listdir(chert):
      train_data_paths.append(os.path.join(chert, rock_image))
      train_labels.append(2)
  elif rock_type =='Diatomite':
    Diatomite = os.path.join(train_DATA_DIR,'Diatomite')
    for rock_image in os.listdir(Diatomite):
      train_data_paths.append(os.path.join(Diatomite,rock_image))
      train_labels.append(3)
  elif rock_type =='olivine-basalt':
    OlivineBasalt = os.path.join(train_DATA_DIR,'olivine-basalt')
    for rock_image in os.listdir(OlivineBasalt):
      train_data_paths.append(os.path.join(OlivineBasalt, rock_image))
      train_labels.append(4)
  elif rock_type =='Siliceous-sinter':
    SiliceousSinter = os.path.join(train_DATA_DIR,'Siliceous-sinter')
    for rock_image in os.listdir(SiliceousSinter):
      train_data_paths.append(os.path.join(SiliceousSinter,rock_image))
      train_labels.append(5)
  elif rock_type =='Conglomerate':
    Conglomerate = os.path.join(train_DATA_DIR,'Conglomerate')
    for rock_image in os.listdir(Conglomerate):
      train_data_paths.append(os.path.join(Conglomerate,rock_image))
      train_labels.append(6)
  elif rock_type =='Clay':
    Clay = os.path.join(train_DATA_DIR,'Clay')
    for rock_image in os.listdir(Clay):
      train_data_paths.append(os.path.join(Clay,rock_image))
      train_labels.append(7)
  elif rock_type =='Basalt':
    Basalt = os.path.join(train_DATA_DIR,'Basalt')
    for rock_image in os.listdir(Basalt):
      train_data_paths.append(os.path.join(Basalt,rock_image))
      train_labels.append(8)

In [14]:
print(len(train_labels))

3687


In [15]:
import random
combined = list(zip(train_data_paths,train_labels))

random.shuffle(combined)
train_data_paths,train_labels = zip(*combined)

train_data_paths,train_labels = list(train_data_paths),list(train_labels)


In [16]:
len(train_data_paths)

3687

In [17]:
len(train_labels)

3687

In [18]:
type(train_data_paths)

list

In [19]:
from torchvision.transforms import transforms
custom_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(
        brightness=0.3,
        contrast=0.3,
        saturation=0.2
    ),
    transforms.ToTensor()
])

In [21]:
from torch.utils.data import DataLoader,Dataset
from PIL import Image

In [22]:
class CustomDataset(Dataset):
  def __init__(self,image_paths,labels,transforms = custom_transform):
    self.images = []
    self.image_paths = image_paths
    self.labels = labels
    self.transforms = transforms
    for img in self.image_paths:
      self.image = Image.open(img).convert('RGB')
      self.images.append(self.image)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):

    return self.transforms(self.images[index]),self.labels[index]


In [23]:
train_dataset = CustomDataset(train_data_paths,train_labels)
train_loader = DataLoader(train_dataset,batch_size=128,shuffle = True)

In [24]:
import torch.nn as nn
import torchvision.models as models
mobilenet_v2 = models.mobilenet_v2(pretrained = True)
#removing the classifier and keeping just the extractor
#mobilenet_v2.classifier = nn.Identity()
#freezing the feature extractor
for param in mobilenet_v2.features.parameters():
  param.requires_grad = False
for param in mobilenet_v2.features[-2:].parameters():
  param.requires_grad = True
for param in mobilenet_v2.classifier.parameters():
  param.requires_grad = True

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 175MB/s]


In [25]:
import torch.optim as optim
learning_rate = 1e-3
epochs = 10
Criterion = nn.CrossEntropyLoss()
optimiezer = optim.Adam(filter(lambda p: p.requires_grad, mobilenet_v2.parameters()),
                        lr= learning_rate)
